In [2]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import roc_auc_score
import shutil
from concurrent.futures import ThreadPoolExecutor
import wandb

In [3]:
wandb.login()

wandb: Currently logged in as: mrahbar-2001 (mrahbar-2001-university-of-isfahan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(" Using device:", device)

 Using device: cuda


In [6]:
# ======= LRF =======
class LRFModel(nn.Module):
    def __init__(self, backbone_name='vit_base_patch16_224', rank=64, num_classes=5):
        super().__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=True)
        in_features = self.backbone.head.in_features
        self.backbone.reset_classifier(0)

        self.low_rank_projection = nn.Sequential(
            nn.Linear(in_features, rank, bias=False),
            nn.ReLU()
        )
        self.classifier = nn.Linear(rank, num_classes)

    def forward_features(self, x):
        return self.backbone.forward_features(x)

    def forward(self, x):
        feats = self.forward_features(x)
        proj = self.low_rank_projection(feats)
        logits = self.classifier(proj)
        return logits

    def get_image_embedding(self, x):
        feats = self.forward_features(x)
        proj = self.low_rank_projection(feats)
        return proj

In [49]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset

class CheXpertDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        """
        پارامترها:
            csv_file (str): مسیر فایل CSV شامل مسیرها و لیبل‌ها
            img_dir (str): مسیر پوشه اصلی تصاویر
            transform (callable, optional): ترنسفورم‌های PyTorch برای اعمال روی تصویر
        """
        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.label_cols = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        # استخراج مسیر نسبی تصویر از فایل CSV
        img_rel_path = self.labels_df.iloc[idx]['Path']

        # حذف پیشوند "CheXpert-v1.0-small/" در صورت وجود
        if img_rel_path.startswith("CheXpert-v1.0-small/"):
            img_rel_path = img_rel_path[len("CheXpert-v1.0-small/"):]

        # ساخت مسیر کامل تصویر
        img_path = os.path.join(self.img_dir, img_rel_path)

        # تلاش برای بارگذاری تصویر، و جایگزینی با تصویر سیاه در صورت خطا
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"[WARNING] Could not load image: {img_path} -- {e}")
            image = Image.new('RGB', (224, 224), (0, 0, 0))  # تصویر جایگزین سیاه

        # اعمال ترنسفورم در صورت وجود
        if self.transform:
            image = self.transform(image)

        # استخراج لیبل‌ها و تبدیل امن به float32 بدون هشدار
        labels = self.labels_df.iloc[idx][self.label_cols]
        labels = labels.infer_objects(copy=False).fillna(0).values.astype('float32')

        return image, labels, img_rel_path


In [50]:
# ====== Transforms ======
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [35]:
# ====== Loaders ======



full_train_df = pd.read_csv('/content/drive/MyDrive/chexpert_data_v2/train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/chexpert_data_v2/valid.csv')

subset_df = full_train_df.sample(frac=0.3, random_state=42).reset_index(drop=True)
subset_df.to_csv("chexpert_30percent.csv", index=False)

source_root = '/content/drive/MyDrive/chexpert_data_v2'
target_root = '/content/chexpert_data_v2_selected'
image_paths = subset_df['Path'].str.replace('CheXpert-v1.0-small/', '', regex=False).tolist()

os.makedirs(target_root, exist_ok=True)
def copy_file(rel_path):
    src = os.path.join(source_root, rel_path)
    dst = os.path.join(target_root, rel_path)
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    try:
        shutil.copy2(src, dst)
    except:
        pass

from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=16) as executor:
    list(tqdm(executor.map(copy_file, image_paths), total=len(image_paths)))



100%|██████████| 67024/67024 [03:25<00:00, 325.80it/s]


In [24]:
!ls /content/drive/MyDrive/chexpert_data_v2/


train  train.csv  valid  valid.csv


In [53]:
train_dataset = CheXpertDataset(
    "chexpert_30percent.csv",
    "/content/chexpert_data_v2_selected",
    transform
)

val_dataset = CheXpertDataset(
    "/content/drive/MyDrive/chexpert_data_v2/valid.csv",
    "/content/drive/MyDrive/chexpert_data_v2",
    transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [54]:
# ====== Training setup ======
model = LRFModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)
scaler = GradScaler()
criterion = nn.BCEWithLogitsLoss(reduction='none')

/tmp/ipython-input-54-4007471735.py:5: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [38]:
wandb.init(
    project="chexpert-lrf-vit",
    name="run-vit-lrf-v1",
    config={
        "lr": 1e-4,
        "batch_size": 128,
        "epochs": 10,
        "model": "ViT + LRF",
        "rank": 64
    }
)

In [39]:


def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for images, labels, _ in tqdm(dataloader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        with autocast():
            outputs = model(images)
            outputs = outputs.mean(dim=1)
            mask = (labels != -1).float()
            loss_raw = criterion(outputs, labels)

            # جلوگیری از تقسیم بر صفر
            if mask.sum() > 0:
                loss = (loss_raw * mask).sum() / mask.sum()
            else:
                loss = loss_raw.mean()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * images.size(0)
    return total_loss / len(dataloader.dataset)


def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, labels, _ in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)
            with autocast():
                outputs = model(images)
                outputs = outputs.mean(dim=1)
                mask = (labels != -1).float()
                loss_raw = criterion(outputs, labels)

                if mask.sum() > 0:
                    loss = (loss_raw * mask).sum() / mask.sum()
                else:
                    loss = loss_raw.mean()
            total_loss += loss.item() * images.size(0)
    return total_loss / len(dataloader.dataset)


In [41]:
# # نمایش 5 نمونه‌ی تصادفی از train_dataset
# import matplotlib.pyplot as plt

# for i in range(5):
#     img, label, path = train_dataset[i]
#     plt.imshow(img.permute(1, 2, 0))  # برای نمایش [C,H,W] → [H,W,C]
#     plt.title(f"Path: {path}\nLabels: {label}")
#     plt.axis('off')
#     plt.show()


In [ ]:
best_val = float('inf')
for epoch in range(10):
    print(f"\n📚 Epoch {epoch+1}/10")
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss = validate(model, val_loader, criterion, device)
    print(f"✅ Train Loss: {train_loss:.4f} | 🧪 Val Loss: {val_loss:.4f}")

    if val_loss < best_val:
        best_val = val_loss
        torch.save(model.state_dict(), "best_lrf_model.pt")
        print("💾 Best model saved!")

    scheduler.step()



📚 Epoch 1/10


  0%|          | 0/1048 [00:00<?, ?it/s]/tmp/ipython-input-39-1551112850.py:8: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
  0%|          | 0/4 [00:00<?, ?it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 25%|██▌       | 1/4 [01:38<04:56, 98.67s/it]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 50%|█████     | 2/4 [03:10<03:08, 94.37s/it]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 75%|███████▌  | 3/4 [04:58<01:41, 101.03s/it]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torc

✅ Train Loss: 0.4231 | 🧪 Val Loss: 0.4721
💾 Best model saved!

📚 Epoch 2/10


  0%|          | 0/1048 [00:00<?, ?it/s]/tmp/ipython-input-39-1551112850.py:8: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
  0%|          | 0/4 [00:00<?, ?it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 25%|██▌       | 1/4 [00:00<00:01,  1.82it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 50%|█████     | 2/4 [00:01<00:01,  1.84it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 75%|███████▌  | 3/4 [00:01<00:00,  1.86it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch

✅ Train Loss: 0.3823 | 🧪 Val Loss: 0.4694
💾 Best model saved!

📚 Epoch 3/10


...

  0%|          | 0/1048 [00:00<?, ?it/s]/tmp/ipython-input-39-1551112850.py:8: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
  0%|          | 0/4 [00:00<?, ?it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 25%|██▌       | 1/4 [00:00<00:01,  1.57it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 50%|█████     | 2/4 [00:01<00:01,  1.69it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 75%|███████▌  | 3/4 [00:01<00:00,  1.78it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch

✅ Train Loss: 0.2265 | 🧪 Val Loss: 0.5412

📚 Epoch 7/10


  0%|          | 0/1048 [00:00<?, ?it/s]/tmp/ipython-input-39-1551112850.py:8: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
  0%|          | 0/4 [00:00<?, ?it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 25%|██▌       | 1/4 [00:00<00:02,  1.43it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 50%|█████     | 2/4 [00:01<00:01,  1.46it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 75%|███████▌  | 3/4 [00:01<00:00,  1.61it/s]/tmp/ipython-input-39-1551112850.py:34: FutureWarning: `torch

✅ Train Loss: 0.1296 | 🧪 Val Loss: 0.7918

📚 Epoch 8/10


  0%|          | 0/1048 [00:00<?, ?it/s]/tmp/ipython-input-39-1551112850.py:8: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
 63%|██████▎   | 656/1048 [05:41<03:34,  1.83it/s]

In [ ]:
# ====== Extract image embeddings and save ======
model.load_state_dict(torch.load("best_lrf_model.pt"))
model.eval()

image_embeddings = {}
with torch.no_grad():
    for images, _, paths in tqdm(val_loader):
        images = images.to(device)
        embs = model.get_image_embedding(images)  # (B, R)
        for path, emb in zip(paths, embs):
            image_embeddings[path] = emb.cpu()

torch.save(image_embeddings, "/content/drive/MyDrive/image_embeddings.pt")
print("✅ Image embeddings saved!")


In [ ]:
# import torch
# import gc

# gc.collect()          # جمع‌آوری حافظه آزاد شده در پایتون
# torch.cuda.empty_cache()  # آزادسازی کش حافظه GPU
